In [72]:
from s3fs.core import S3FileSystem
import pandas as pd
import os
import trino
from dotenv import load_dotenv


In [55]:
# Leemos el archivo y guardamos en formato parquet
epf = pd.read_csv("../data/base-gastos-ix-epf.csv", encoding = "latin1",  sep=";", decimal=","   ) 

# Guardamos en parquet
epf = epf[["glosa_ccif", "d", "g", "c", "sc", "p", "gasto"]]
epf.to_parquet("../data/base-gastos-ix-epf.parquet")
 

In [56]:
# Agregar credenciales
access = "capacitacion"
secret = "capacitacion"
s3 = S3FileSystem(
    key=access,
    secret=secret,
    endpoint_url="http://10.91.160.65:9000"
)

In [57]:
# Guardamos en minio 
bucket = "capacitacion-lago-datos"
file = "base-gastos-ix-epf.parquet"
source_folder = "../data"

minio_path = f"{bucket}/data/epf_lehmann/{file}"
local_path = f"{source_folder}/{file}"
s3.put_file(rpath=minio_path,lpath=local_path)


In [58]:
# Comprobar que esté el archivo
s3.ls(f"{bucket}/data/epf_lehmann/")

['capacitacion-lago-datos/data/epf_lehmann/base-gastos-ix-epf.parquet']

In [59]:
# Leemos el archivo
parquet_example = pd.read_parquet(minio_path, filesystem=s3)

In [60]:
# Miramos las primeras filas
parquet_example.head()


,glosa_ccif,d,g,c,sc,p,gasto
0,PAN CORRIENTE A GRANEL,1.0,1.0,1.0,3.0,1.0,23914.285714
1,MASAS SALADAS SIN RELLENO Y SIN AGREGADOS PARA...,1.0,1.0,1.0,3.0,12.0,27678.571429
2,CARNE DE VACUNO CORTE POSTA,1.0,1.0,2.0,2.0,4.0,23250.000000
3,OTROS CORTES DE CARNE DE VACUNO,1.0,1.0,2.0,2.0,6.0,36535.714286
4,PECHUGA DE POLLO,1.0,1.0,2.0,2.0,18.0,7750.000000


In [61]:

# Cargamos variables de ambiente 
load_dotenv() 

# Setear credenciales
conn = trino.dbapi.connect(
        host=os.environ["host"], # cambiar por IP
        port=os.environ["port"], # cambiar puerto
        user=os.environ["user"], # cambiar usuario
        catalog=os.environ["catalog"], # cambiar catalogo
        schema = os.environ["schema"] # cambiar esquema
)


In [62]:
table_name = "klaus"
bucket = "capacitacion-lago-datos"
folder = "data/epf_lehmann"

# Esta query elimina la tabla, en caso de que ya exista
query_drop = f"drop table if exists minio.capacitacion.{table_name}"

# La query lee todo lo que hay en un directorio y lo convierte en una tabla de trino 
query = f"""
    CREATE TABLE minio.capacitacion.{table_name} (
        glosa_ccif VARCHAR,
        d DOUBLE,
        g DOUBLE, 
        c DOUBLE,
        sc DOUBLE,
        p DOUBLE,
        gasto DOUBLE
        
    )
    WITH (
        format = 'PARQUET', 
        external_location = 's3a://capacitacion-lago-datos/{folder}'
    )
    """


In [63]:
# Ejecutamos la query para borrar
cursor = conn.cursor()
cursor.execute(query_drop)

# ejecutamos la query para crear tabla 
cursor.execute(query)


In [70]:
# Contemos el número de filas de la tabla
query = "select count(*) from klaus"
cursor.execute(query)
data = cursor.fetchall()
data


[[970773]]

In [71]:
query = "select * from klaus limit 10"
cursor.execute(query)
data = cursor.fetchall()
data

[['PAN CORRIENTE A GRANEL', 1.0, 1.0, 1.0, 3.0, 1.0, 23914.2857142857],
 ['MASAS SALADAS SIN RELLENO Y SIN AGREGADOS PARA COCINAR',
  1.0,
  1.0,
  1.0,
  3.0,
  12.0,
  27678.5714285714],
 ['CARNE DE VACUNO CORTE POSTA', 1.0, 1.0, 2.0, 2.0, 4.0, 23250.0],
 ['OTROS EMBUTIDOS FRESCOS', 1.0, 1.0, 2.0, 5.0, 5.0, 11071.4285714286],
 ['ATÃ\x9aN EN CONSERVA', 1.0, 1.0, 3.0, 3.0, 2.0, 26571.4285714286],
 ['QUESO GOUDA', 1.0, 1.0, 4.0, 5.0, 1.0, 8857.14285714286],
 ['OTROS CORTES DE CARNE DE VACUNO',
  1.0,
  1.0,
  2.0,
  2.0,
  6.0,
  36535.7142857143],
 ['PECHUGA DE POLLO', 1.0, 1.0, 2.0, 2.0, 18.0, 7750.0],
 ['CECINAS Y FIAMBRES DE POLLO COCIDOS',
  1.0,
  1.0,
  2.0,
  3.0,
  5.0,
  2214.28571428571],
 ['VIENESAS TRADICIONALES', 1.0, 1.0, 2.0, 5.0, 1.0, 2214.28571428571]]

In [69]:
# Query group by
query = "select d,  count(*) as frecuencia from klaus group by d order by d"
cursor.execute(query)
data = cursor.fetchall()
data



[[1.0, 442763],
 [2.0, 15121],
 [3.0, 54368],
 [4.0, 78987],
 [5.0, 66853],
 [6.0, 45844],
 [7.0, 48815],
 [8.0, 47630],
 [9.0, 37553],
 [10.0, 11871],
 [11.0, 39902],
 [12.0, 33207],
 [13.0, 47857],
 [None, 2]]